# Manipulate `Records` after advancing year

Sample run adding 10% to wages (`e00200`) in tax year 2018.

## Setup

### Imports

In [1]:
import taxcalc as tc
import pandas as pd
import numpy as np
import os

In [2]:
tc.__version__

'0.20.1'

## Generate data

### Load data

Generate a set of normal CPS records for 2018 using `Calculator`, then extract the dataframe.

In [3]:
base_calc = tc.Calculator(records=tc.Records.cps_constructor(), 
                          policy=tc.Policy(), verbose=False)
base_calc.advance_to_year(2018)

In [4]:
cps_raw_cols = pd.read_csv(
    os.path.join(tc.Records.CUR_PATH, 'cps.csv.gz')).columns

In [5]:
df = base_calc.dataframe(list(cps_raw_cols))

### Manipulate

In [6]:
df['e00200_orig'] = df.e00200

In [7]:
df['e00200'] = df.e00200 * 1.1
df['e00200p'] = df.e00200p * 1.1
df['e00200s'] = df.e00200s * 1.1

### Clean data

Fix `e00900` pending https://github.com/open-source-economics/Tax-Calculator/issues/2024.

In [8]:
df['e00900'] = df.e00900p + df.e00900s

## Calculate

Run Tax-Calculator.

Adapt [`cps_constructor` code](https://github.com/open-source-economics/Tax-Calculator/blob/master/taxcalc/records.py#L187).

In [9]:
recs = tc.Records(data=df, 
                  start_year=2018, 
                  weights=tc.Records.CPS_WEIGHTS_FILENAME,
                  adjust_ratios=tc.Records.CPS_RATIOS_FILENAME)

In [10]:
calc = tc.Calculator(records=recs, policy=tc.Policy())
calc.calc_all()

You loaded data for 2018.
Your data include the following unused variables that will be ignored:
  e00200_orig
Tax-Calculator startup automatically extrapolated your data to 2018.


## Verify

Fields should match df.

In [11]:
calc_df = calc.dataframe(['e00200'])

In [12]:
df.e00200.equals(calc_df.e00200)

True